### Once the best model is chosen based on the experiments on the computational cluster, the model is used to impute the missing data here

In [ ]:
import os
import pickle
import json
import math

import pandas as pd
from energyemissionsregio.config import DATA_PATH
from energyemissionsregio.utils import get_confidence_level

In [ ]:
cwd = os.getcwd()

In [ ]:
vars_details = [
    {"var_to_impute": "es_utilized_agricultural_area",
                    "best_corr_threshold": 0.1, 
                    "r2": 0.89},

                {"var_to_impute": "es_number_of_dwellings",
                    "best_corr_threshold": 0.1, 
                    "r2":  0.97},
                ]
                

In [ ]:
def get_x_vars_df(predictor_vars):
    X_vars_df = None

    for var_name in predictor_vars:
        
        _df = pd.read_csv(
            os.path.join(DATA_PATH, f"{var_name}.csv")
        )
        _df = _df[_df["region_code"].str.startswith(("ES", "DE"))][["region_code", "value"]].copy()

        _df = _df.fillna(0) # filling NAs for point vars. Non-point vars have no NAs in Spain

        _df.rename(columns={"value": var_name}, inplace=True)

        if X_vars_df is not None:
            X_vars_df = pd.merge(X_vars_df, _df, on="region_code", how="outer")
        else:
            X_vars_df = _df

    return X_vars_df

In [ ]:
for vars_detail_dict in vars_details:
    var_to_impute = vars_detail_dict["var_to_impute"]
    best_corr_threshold = vars_detail_dict["best_corr_threshold"]
    r2 = vars_detail_dict["r2"]

    imputed_value_confidence_level = get_confidence_level(r2)

    print(var_to_impute)

    with open(
            os.path.join(
                cwd, "..", "..", "data", 
                "missing_value_imputation",
                  "predictor_vars", 
                  f"{var_to_impute}_{best_corr_threshold}corr.json"
            )
        ) as f:
            predictor_vars = tuple(json.load(f))

    X_vars_df = get_x_vars_df(predictor_vars)

    y_var_df = pd.read_csv(
        os.path.join(DATA_PATH, f"{var_to_impute}.csv")
    )

    y_var_df.rename(columns={"value": var_to_impute}, inplace=True)
    y_var_df.drop(columns="year", inplace=True)

    final_df = pd.merge(X_vars_df, y_var_df, on="region_code", how="left")

    # fit the model on missing data
    missing_values_df = final_df.copy()
    missing_values_df = missing_values_df[missing_values_df[
                            var_to_impute].isna()].drop(columns=[var_to_impute])

    missing_values_df_no_reg_code = missing_values_df.drop(columns=["region_code"])

    file_path = os.path.join(cwd, "..", "..", "data", 
                            "missing_value_imputation", 
                            "models", 
                            f"{var_to_impute}_xgb_{best_corr_threshold}corr.pkl")
    
    with open(file_path, 'rb') as f:
        model = pickle.load(f)
 
    y_pred = model.predict(missing_values_df_no_reg_code)

    missing_values_df['imputed_values'] = y_pred

    assert missing_values_df["imputed_values"].min() >= 0

    # filling missing data and assign value_confidence_level
    imputed_df = final_df[["region_code", var_to_impute]].copy()
    imputed_df.rename(columns={var_to_impute: "value"}, inplace=True) 

    imputed_df["value_confidence_level"] = 5 # VERY HIGH 

    for idx, row in final_df.iterrows():
        region_code = row["region_code"]

        if math.isnan(row[var_to_impute]):
            impute_value = missing_values_df[missing_values_df[
                                "region_code"] == region_code][["imputed_values"]].values.item()
            
            # round off values (if int type is desired)
            if "number" in var_to_impute:
                imputed_df.loc[idx, "value"] = int(impute_value)
            else:
                imputed_df.loc[idx, "value"] = impute_value

            imputed_df.loc[idx, "value_confidence_level"] = imputed_value_confidence_level

    imputed_df.to_csv(os.path.join(
            cwd, "..", "..", "data", "imputed_data", f"{var_to_impute}.csv"
        ), index=False)